<a href="https://colab.research.google.com/github/DenShum/A-B-test/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22A_B_%D1%82%D0%B5%D1%81%D1%82_4_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Выберите две особенности в данных из приведенных ниже, сформулируйте для них гипотезы по всем правилам:

● В период Surge видим более низкую price-to-order конверсию

● Для устройств Android наблюдаем более высокие юзерские отмены

● Если Заказ из центра, то видим более низкие юзерские отмены


● Если Comfort & Business классы, то более низкие юзерские отмены, более длинные поездки

Разбейте юзеров на группы с помощью рандома, например, по следующему алгориму:

● Добавьте новую колонку, где каждому юзеру присвойте значение функции RAND() - выдает случайное число от 0 до 1

● В зависимости от значения rand - поместите пользователя в группу 1 или 2. (если rand >0.5, то 1, иначе 2)

Проверьте на однородность с помощью t-теста (встроенной функией excel)
Важно: функция работает только на выборках одинакового размера. Убедитесь что группы 1 и 2 одинаковы по количеству юзеров. Для выравнивания можно исключить некоторое кол-во юзеров в большей группе.



In [ ]:
import pandas as pd
df = pd.read_csv('/content/Netology_A_B-testing_HW_1.csv')
print(df.shape) #количество стр. и столб.
df.info()
# df = df.apply(pd.to_numeric,errors='coerce') #меняем тип данных на числовые
# df.describe().T

(101500, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101500 entries, 0 to 101499
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         101500 non-null  int64  
 1   date               101500 non-null  object 
 2   user_id            101500 non-null  int64  
 3   hour               101500 non-null  int64  
 4   app_opened         101500 non-null  int64  
 5   price_seen         101500 non-null  int64  
 6   order_made         101500 non-null  int64  
 7   surge              91431 non-null   object 
 8   ride_completed     101500 non-null  int64  
 9   user_cancelled     101500 non-null  int64  
 10  age                101500 non-null  int64  
 11  os                 101500 non-null  object 
 12  city_center_order  101500 non-null  int64  
 13  order_class        101500 non-null  object 
 14  distance           91431 non-null   float64
 15  rfm                101500 non-null  ob

In [ ]:
df.head(10).T

In [ ]:
df['surge'] = df['surge'].fillna('no surge')# заменяем NaN на 'no surge' в кол.surge
df.loc[(df.distance > 0), 'order_made'] = 1 # ставим 1 в кол.'order_made', если в кол.distance есть расстояние
df.head(10).T # ОЧИЩЕННЫЙ ДАТАФРЕЙМ !!!

,0,1,2,3,4,5,6,7,8,9
Unnamed: 0,0,1,2,3,4,5,6,7,8,9
date,2020-05-21,2020-05-23,2020-05-20,2020-05-19,2020-05-23,2020-05-21,2020-05-24,2020-05-22,2020-05-23,2020-05-23
user_id,867689,752172,486559,304024,139420,139455,416988,682337,931492,662775
hour,12,5,15,0,0,5,0,2,20,17
app_opened,1,1,1,1,1,1,1,1,1,1
price_seen,1,1,1,1,1,0,1,1,0,1
order_made,1,1,1,1,1,0,1,1,0,1
surge,no surge,no surge,no surge,no surge,no surge,no surge,no surge,no surge,no surge,surge
ride_completed,1,1,1,1,1,0,0,1,0,1
user_cancelled,0,0,0,0,0,0,0,0,0,0


In [ ]:
# добавляем кол.randNumCol и заполняем ее случ.знач. : 0 или 1, по строкам df
import numpy as np
import pandas as pd
df['randNumCol'] = np.random.choice([0, 1], df.shape[0]) 
df.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
date,2020-05-21,2020-05-23,2020-05-20,2020-05-19,2020-05-23,2020-05-21,2020-05-24,2020-05-22,2020-05-23,2020-05-23,2020-05-23,2020-05-24,2020-05-21,2020-05-19,2020-05-21,2020-05-19,2020-05-23,2020-05-21,2020-05-19,2020-05-20
user_id,867689,752172,486559,304024,139420,139455,416988,682337,931492,662775,464055,370093,811050,786923,382672,893154,990097,621723,393096,320419
hour,12,5,15,0,0,5,0,2,20,17,20,12,4,23,2,6,2,13,23,3
app_opened,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
price_seen,1,1,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,1,1
order_made,1,1,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,1,1
surge,no surge,no surge,no surge,no surge,no surge,no surge,no surge,no surge,no surge,surge,surge,no surge,no surge,no surge,no surge,no surge,no surge,no surge,surge,no surge
ride_completed,1,1,1,1,1,0,0,1,0,1,1,1,0,0,1,1,1,0,0,1
user_cancelled,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.randNumCol.value_counts()# уникальные значения по колонке randNumCol

0    50763
1    50737
Name: randNumCol, dtype: int64

In [ ]:
df0 = df[df.randNumCol==0] #отфильтруем df по randNumCol==0,выборка А
# выбираем нужное количество строк, чтоб выборки А(знач."0") и В(знач."1") были одинаковыми
df0_ = df0.iloc[0:50719] 
df0_.randNumCol.value_counts()


0    50719
Name: randNumCol, dtype: int64

In [ ]:
df1 = df[df.randNumCol==1] #отфильтруем df по randNumCol==1,выборка В
df1.randNumCol.value_counts()

1    50737
Name: randNumCol, dtype: int64

In [ ]:
 #находим средние значения в выборках по столбцу order_made
import scipy.stats as stats
m1, m2 = df0_.order_made.mean(),df1.order_made.mean()
print(m1,m2, end='')

0.9001951931228928 0.9013934603937954

In [ ]:
scal0 = np.std(df0_.order_made) #среднеквадратичное (стандартное) отклонение выборка А
scal1 = np.std(df1.order_made) #среднеквадратичное (стандартное) отклонение выборка В
print(scal0,scal1)

0.2997395659591989 0.29813300715134683


In [ ]:
A = df0_[['order_made']]  # выборка А с одним столбцом order_made
B = df1[['order_made']] # выборка B с одним столбцом order_made
t, p_value = stats.ttest_ind(A, B) #Критерий Стьюдента (t-test)
print(t, p_value)
print(stats.ttest_ind(A, B, equal_var = False))

[-0.63837855] [0.52322873]
Ttest_indResult(statistic=array([-0.63837794]), pvalue=array([0.52322913]))


т.к. Ttest = -0,638 меньше табличного значения 1.960, то статистически значимых различий нет

pvalue > 0,05 ,следовательно статистически значимых различий нет (для уровня значимости 5%)
